# almond-spark

* Based [ammonite-spark](https://github.com/alexarchambault/ammonite-spark)
* Works for any Spark version >= 2.0
* Currently support for local, standalone and yarn clusters

In [8]:
import $ivy.`org.apache.spark::spark-sql:2.4.3` // Or use any other 2.x version here
import $ivy.`sh.almond::almond-spark:0.5.0`

import org.apache.spark.sql._, org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}
def sc = spark.sparkContext

Getting spark JARs
Creating SparkSession


Spark UI

import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                              


import org.apache.spark.sql._, org.apache.log4j.{Level, Logger}

spark: SparkSession = org.apache.spark.sql.SparkSession@2c757b23
defined function sc

## Load a DataFrame

In [9]:
import spark.implicits._
val titanic = spark
  .read
  .format("csv")
  .option("inferSchema", "true")
  .option("header", "true")
  .load("titanic.csv")

load at cmd8.sc:7

1 / 1

load at cmd8.sc:7

2 / 2

import spark.implicits._

titanic: DataFrame = [Survived: int, Pclass: int ... 6 more fields]

## Show as text

In [10]:
titanic.show()

show at cmd9.sc:1

1 / 1

+--------+------+--------------------+------+----+-----------------------+-----------------------+-------+
|Survived|Pclass|                Name|   Sex| Age|Siblings/Spouses Aboard|Parents/Children Aboard|   Fare|
+--------+------+--------------------+------+----+-----------------------+-----------------------+-------+
|       0|     3|Mr. Owen Harris B...|  male|22.0|                      1|                      0|   7.25|
|       1|     1|Mrs. John Bradley...|female|38.0|                      1|                      0|71.2833|
|       1|     3|Miss. Laina Heikk...|female|26.0|                      0|                      0|  7.925|
|       1|     1|Mrs. Jacques Heat...|female|35.0|                      1|                      0|   53.1|
|       0|     3|Mr. William Henry...|  male|35.0|                      0|                      0|   8.05|
|       0|     3|     Mr. James Moran|  male|27.0|                      0|                      0| 8.4583|
|       0|     1|Mr. Timothy J McC...

## Let's make the output a bit nicer

In [11]:
implicit class RichDF(val df: DataFrame) {
  def showHTML(limit:Int = 20) = {
    import xml.Utility.escape
    val data = df.take(limit)
    val header = df.schema.fieldNames.toSeq
    val rows: Seq[Seq[String]] = data.map { row =>
      row.toSeq.map { cell =>
        cell match {
          case null => "null"
          case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
          case array: Array[_] => array.mkString("[", ", ", "]")
          case seq: Seq[_] => seq.mkString("[", ", ", "]")
          case _ => cell.toString
        }
      }: Seq[String]
    }

    publish.html(s"""
    <div>
      <table border="1" class="dataframe">
      <thead>
        <tr>
        ${header.map(h => s"<th>${escape(h)}</th>").mkString}
        </tr>
         </thead>
         <tbody>
        ${rows.map { row =>
          s"<tr>${row.map { c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
        }.mkString
        }
        </tbody>
      </table>
    </div>""")
  }
}

defined class RichDF

In [12]:
titanic.showHTML(8)

take at cmd10.sc:4

1 / 1

Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.25
1,1,Mrs. John Bradley (Florence Briggs Thayer) Cumings,female,38.0,1,0,71.2833
1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.925
1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1
0,3,Mr. William Henry Allen,male,35.0,0,0,8.05
0,3,Mr. James Moran,male,27.0,0,0,8.4583
0,1,Mr. Timothy J McCarthy,male,54.0,0,0,51.8625
0,3,Master. Gosta Leonard Palsson,male,2.0,3,1,21.075


# Let's try some visualization

In [13]:
import $ivy.`org.vegas-viz::vegas-spark:0.3.11`
import vegas._, vegas.data.External._, vegas.sparkExt._

Vegas("Titanic Survivors").
  withDataFrame(titanic).
  mark(Bar).
  encodeY("*", aggregate=AggOps.Count, axis=Axis(title="Number of People", grid=false)).
  encodeColumn("Pclass", Ord, scale=Scale(padding=10.0), axis=Axis(orient=Orient.Bottom, axisWidth=1.0, offset= -8.0)).
  encodeX("Survived", Nominal, scale=Scale(bandSize = 16.0), hideAxis=true).
  encodeColor("Survived", Nominal, scale=Scale(rangeNominals=List("red", "green"))).
  configFacet(cell=CellConfig(strokeWidth = 0)).
  configCell(height=400).
  show

count at package.scala:14

1 / 1

count at package.scala:14

1 / 1

collect at package.scala:16

1 / 1

<iframe id="frame-vegas-5313319f-9bb5-473c-b256-71fd86da8ef8" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-5313319f-9bb5-473c-b256-71fd86da8ef8'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "mark" : "bar",
 "encoding" : {
 "column" : {
 "axis" : {
 "orient" : "bottom",
 "axisWidth" : 1.0,
 "offset" : -8.0
 },
 "scale" : {
 "padding" : 10.0
 },
 "field" : "Pclass",
 "type" : "ordinal"
 },
 "x" : {
 "axis" : false,
 "scale" : {
 "bandSize" : 16.0
 },
 "field" : "Survived",
 "type" : "nominal"
 },
 "y" : {
 "axis" : {
 "title" : "Number of People",
 "grid" : false
 },
 "field" : "*",
 "aggregate" : "count"
 },
 "color" : {
 "scale" : {
 "range" : [
 "red",
 "green"
 ]
 },
 "field" : "Survived",
 "type" : "nominal"
 }
 },
 "description" : "Titanic Survivors",
 "data" : {
 "values" : [
 {
 "Name" : "Mr. Owen Harris Braund",
 "Siblings/Spouses Aboard" : 1,
 "Pclass" : 3,
 "Age" : 22.0,
 "Fare" : 7.25,
 "Parents/Children Aboard" : 0,
 "Survived" : 0,
 "Sex" : "male"
 },
 {
 "Name" : "Mrs. John Bradley (Florence Briggs Thayer) Cumings",
 "Siblings/Spouses Aboard" : 1,
 "Pclass" : 1,
 "Age" : 38.0,
 "Fare" : 71.2833,
 "Parents/Children Aboard" : 0,
 "Survived" : 1,
 "Sex" : "female"
 },
 {
 "Name" : "Miss. Laina Heikkinen",
 "Siblings/Spouses Aboard" : 0,
 "Pclass" : 3,
 "Age" : 26.0,
 "Fare" : 7.925,
 "Parents/Children Aboard" : 0,
 "Survived" : 1,
 "Sex" : "female"
 },
 {
 "Name" : "Mrs. Jacques Heath (Lily May Peel) Futrelle",
 "Siblings/Spouses Aboard" : 1,
 "Pclass" : 1,
 "Age" : 35.0,
 "Fare" : 53.1,
 "Parents/Children Aboard" : 0,
 "Survived" : 1,
 "Sex" : "female"
 },
 {
 "Name" : "Mr. William Henry Allen",
 "Siblings/Spouses Aboard" : 0,
 "Pclass" : 3,
 "Age" : 35.0,
 "Fare" : 8.05,
 "Parents/Children Aboard" : 0,
 "Survived" : 0,
 "Sex" : "male"
 },
 {
 "Name" : "Mr. James Moran",
 "Siblings/Spouses Aboard" : 0,
 "Pclass" : 3,
 "Age" : 27.0,
 "Fare" : 8.4583,
 "Parents/Children Aboard" : 0,
 "Survived" : 0,
 "Sex" : "male"
 },
 {
 "Name" : "Mr. Timothy J McCarthy",
 "Siblings/Spouses Aboard" : 0,
 "Pclass" : 1,
 "Age" : 54.0,
 "Fare" : 51.8625,
 "Parents/Children Aboard" : 0,
 "Survived" : 0,
 "Sex" : "male"
 },
 {
 "Name" : "Master. Gosta Leonard Palsson",
 "Siblings/Spouses Aboard" : 3,
 "Pclass" : 3,
 "Age" : 2.0,
 "Fare" : 21.075,
 "Parents/Children Aboard" : 1,
 "Survived" : 0,
 "Sex" : "male"
 },
 {
 "Name" : "Mrs. Oscar W (Elisabeth Vilhelmina Berg) Johnson",
 "Siblings/Spouses Aboard" : 0,
 "Pclass" : 3,
 "Age" : 27.0,
 "Fare" : 11.1333,
 "Parents/Children Aboard" : 2,
 "Survived" : 1,
 "Sex" : "female"
 },
 {
 "Name" : "Mrs. Nicholas (Adele Achem) Nasser",
 "Siblings/Spouses Aboard" : 1,
 "Pclass" : 2,
 "Age" : 14.0,
 "Fare" : 30.0708,
 "Parents/Children Aboard" : 0,
 "Survived" : 1,
 "Sex" : "female"
 },
 {
 "Name" : "Miss. Marguerite Rut Sandstrom",
 "Siblings/Spouses Aboard" : 1,
 "Pclass" : 3,
 "Age" : 4.0,
 "Fare" : 16.7,
 "Parents/Children Aboard" : 1,
 "Survived" : 1,
 "Sex" : "female"
 },
 {
 "Name" : "Miss. Elizabeth Bonnell",
 "Siblings/Spouses Aboard" : 0,
 "Pclass" : 1,
 "Age" : 58.0,
 "Fare" : 26.55,
 "Parents/Children Aboard" : 0,
 "Survived" : 1,
 "Sex" : "female"
 },
 {
 "Name" : "Mr. William Henry Saundercock",
 "Siblings/Spouses Aboard" : 0,
 "Pclass" : 3,
 "Age" : 20.0,
 "Fare" : 8.05,
 "Parents/Children Aboard" : 0,
 "Survived" : 0,
 "Sex" : "male"
 },
 {
 "Name" : "Mr. Anders Johan Andersson",
 "Siblings/Spouses Aboard" : 1,
 "Pclass" : 3,
 "Age" : 39.0

import $ivy.$                                  

import vegas._, vegas.data.External._, vegas.sparkExt._

